In [1]:
from textblob.classifiers import NaiveBayesClassifier
import json
# SQL Alchemy
from sqlalchemy import create_engine
import pandas as pd
import pymysql
pymysql.install_as_MySQLdb()
from textblob import TextBlob
import config

In [2]:
#connection to database
engine = create_engine("mysql://root:" + config.root_pwd + "@localhost/simpsons")
conn = engine.connect()

In [3]:
#query all data for the top 4 characters limited to 2500 results
simp_df=pd.read_sql('(select s.normalized_text as text, "Marge" as label from s_lines s join episodes e on s.episode_id=e.episode_id where s.character_id=1 limit 1000, 2500) union (select s.normalized_text as text, "Homer" as label from s_lines s join episodes e on s.episode_id=e.episode_id where s.character_id=2 limit 1000, 2500) union (select s.normalized_text as text, "Bart" as label from s_lines s join episodes e on s.episode_id=e.episode_id where s.character_id=8 limit 1000, 2500) union (select s.normalized_text as text, "Lisa" as label from s_lines s join episodes e on s.episode_id=e.episode_id where s.character_id=9 limit 1000, 2500)', conn)
Simp_dict = simp_df.to_dict(orient="records")
simp_df

,text,label
0,homer ive been talking to lisa and im concerne...,Marge
1,homer please,Marge
2,well ill tell you what why dont we invite mr b...,Marge
3,sure,Marge
4,no,Marge
5,yes,Marge
6,lisa tell your father,Marge
7,homer you are not allowed to have hurt feeling...,Marge
8,homie did you straighten everything out,Marge
9,homer you promised,Marge


In [4]:
#write to json file
with open('data/ML2.json', 'w') as fp:
    json.dump(Simp_dict, fp)

In [5]:
#read from exported json file
with open('data/ML2.json', 'r') as fp:
    cl = NaiveBayesClassifier(fp, format="json")

In [15]:
#classify who said the line in parethesis
prob_dist = cl.prob_classify('uh oh school bus gotta go')
prob_dist.max()

'Bart'

In [16]:
#print out the percentage of the character it predicted
print(round(prob_dist.prob('Marge'), 2))
print(round(prob_dist.prob('Homer'), 2))
print(round(prob_dist.prob('Bart'), 2))
print(round(prob_dist.prob('Lisa'), 2))

0.0
0.03
0.9
0.06


In [11]:
#convert the line to blob and classify again
blob = TextBlob('does this work', classifier=cl)
blob.classify()

'Homer'